In [3]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from ultralytics import YOLO

In [4]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.147 🚀 Python-3.11.3 torch-2.0.1 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 291.6/460.4 GB disk)


In [35]:
#!yolo task=detect mode=train model=yolov8n.pt data=../all_datasets/data_V/data.yaml epochs=30 imgsz=640 save=True

In [26]:
from PIL import Image
imageFromCDataset = "../localTesting/video3_9.jpg"
imageFromVDataset = "../localTesting/fromModelVValid.jpg"
independentCameraView = "../localTesting/independent_camera.jpeg"

In [27]:
modelV = YOLO("models/V/weights/V.pt")
modelC = YOLO("models/C/weights/C.pt")

In [28]:
# for an image from modelC's dataset (camera view),

import torch
with torch.no_grad():
    resultsV = modelV(imageFromCDataset, save=True) # around 0.79 (trained with 'vest' images)
    resultsC = modelC(imageFromCDataset, save=True) # around 0.86 (trained with camera view)



image 1/1 /Users/dogukan/repos/safety_ai/object_detector_model_training/../localTesting/video3_9.jpg: 640x224 1 vest, 35.0ms
Speed: 1.1ms preprocess, 35.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 224)
Results saved to /opt/homebrew/runs/detect/predict9

image 1/1 /Users/dogukan/repos/safety_ai/object_detector_model_training/../localTesting/video3_9.jpg: 640x224 1 helmet, 1 vest, 35.5ms
Speed: 1.0ms preprocess, 35.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 224)
Results saved to /opt/homebrew/runs/detect/predict10


In [23]:
# for an image from modelV's dataset (camera view),

import torch
with torch.no_grad():
    resultsV = modelV(imageFromVDataset, save=True) # detected 3 vests
    resultsC = modelC(imageFromVDataset, save=True) # vests (3) are not detected


image 1/1 /Users/dogukan/repos/safety_ai/object_detector_model_training/../localTesting/fromModelVValid.jpg: 512x640 3 vests, 78.6ms
Speed: 2.3ms preprocess, 78.6ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
Results saved to /opt/homebrew/runs/detect/predict7

image 1/1 /Users/dogukan/repos/safety_ai/object_detector_model_training/../localTesting/fromModelVValid.jpg: 512x640 2 vests, 59.2ms
Speed: 2.0ms preprocess, 59.2ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)
Results saved to /opt/homebrew/runs/detect/predict8


tensor([0.7937, 0.7465, 0.3282])


In [30]:
print(resultsV[0].boxes.conf)

tensor([0.7937, 0.7465, 0.3282])


In [32]:

# a prediction on an external image
import torch
with torch.no_grad():
    resultsV = modelV(independentCameraView, save=True)
    resultsC = modelC(independentCameraView, save=True) 

# model V detected 8/10 vests for a new camera view; with confidences between 0.4-0.79 (hiqh quality, not cropped image)
# model C not detected any...


image 1/1 /Users/dogukan/repos/safety_ai/object_detector_model_training/../localTesting/independent_camera.jpeg: 352x640 8 vests, 65.5ms
Speed: 3.9ms preprocess, 65.5ms inference, 2.2ms postprocess per image at shape (1, 3, 352, 640)
Results saved to /opt/homebrew/runs/detect/predict9

image 1/1 /Users/dogukan/repos/safety_ai/object_detector_model_training/../localTesting/independent_camera.jpeg: 352x640 (no detections), 49.2ms
Speed: 1.8ms preprocess, 49.2ms inference, 0.2ms postprocess per image at shape (1, 3, 352, 640)
Results saved to /opt/homebrew/runs/detect/predict10


In [34]:
# these results implies that a model with a combined dataset (C and V) would work more efficient in general. 
# ModelC by it's own, is not enough. 
# ModelV works fine in general, but since this is the main usage logic of our program, it should be tested on cropped camera images as well.